In [122]:
import os
import re
import mikeio
import numpy as np
import pandas as pd
import datetime as dt
from res1d_v5 import *
import pickle
import plotly.express as px
from IPython.display import clear_output

Brainstorm

Add flood zoomable floodmap
Add 1s result timestep for pump stations


In [162]:
#General variables

period_specs = []
period_specs.append('DWF')
for i in range(1,6):
    period_specs.append('WWF' + str(i))
period_specs

['DWF', 'WWF1', 'WWF2', 'WWF3', 'WWF4', 'WWF5']

In [65]:
#Model specific variables
model_area = "NSSA"

rolling_average = False

#Only for debug then remove
gauges = ['HL14']

calibration_sheet = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Calibration_Master.xlsm"
rainfall_dfs0_file = ""

map_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Report_Maps"
output_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\03. CALIB_REPORT\Latest_Backup49-Regenerate_Accumulation\Jupyter_Reports"
result_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\07_CALIBRATION\02. WWF_CALIBRATION\06. CALIB_RESULTS\Latest_Backup49"

dfs0_folders = []
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS")
dfs0_folders.append(r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\PS")

In [56]:
gauges = pd.read_excel(calibration_sheet,sheet_name="Gauges")
gauges = gauges.loc[:, :'Level Type']
gauges = gauges[gauges.Model.str.contains(model_area)]

periods = pd.read_excel(calibration_sheet,sheet_name="Periods")
periods = periods[periods.Model.str.contains(model_area)]

report_text = pd.read_excel(calibration_sheet,sheet_name="Report_Text")
report_text = report_text[report_text.Model.notna()]
report_text = report_text[report_text.Model.str.contains(model_area)]
report_text


,Zone,DWF Text,WWF Text,Issue Text,Recommendation Text,Model
172,FA2,The flow measurement is spiky. After DWF cali...,FA2 shows good match in total volume and peak ...,Manhole spilling is shown in simulation but ca...,check with local municipality about manhole sp...,NSSA-E
173,LY7,NaN,LY7 combines the flows in LY1 and LY2. The num...,It was found that the previously described con...,The pump setup in the model to match the contr...,NSSA-E
174,MK20,"Due to the lack of reliability with the MK20, ...",DWF calibration is further improved before WWF...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,"Recommend that metering be repeated, at a loca...",NSSA-E
175,MK3,"Due to the lack of reliability with the MK20, ...",MK3 is calibrated together with NV8.\nIn valid...,GWI of d/s MK3 meter < GWI of u/s MK20 meter. ...,Recommend that either the MV MK3 be fixed and ...,NSSA-E
176,NV1,"After DWF calibration, peak level difference a...",NV1 is inconsistent with upstream and downstre...,NaN,NaN,NSSA-E
178,NVLS5,NV19 was used to calibrate for DWF. The per ca...,NVLS1 is located at the outlet chamber. The to...,NVLS5 has no level data.,NaN,NSSA-E
179,NV13,NaN,NV13 has a good match for the October and the ...,NV13 has no data for the two last events.,NaN,NSSA-E
180,NV19,The per capita and per area loading rates have...,NV19 is not calibrated in WWF.,The larger discrepancy between the peak flow r...,NaN,NSSA-E
181,NV20,The per capita and per area loading rates have...,NV20 has a good match in total volumes and pea...,The flow proportionality between NV20 and its ...,"For future metering plans, implementation and ...",NSSA-E
182,NV21,"In terms of field measurements, ADWF of the u/...",NV21 is downstream of New Lynn Siphon and is u...,ADWF of the u/s NV19 is 85.9 L/s which is larg...,Tributary to NV19 and NV21 may benefit from fu...,NSSA-E


In [70]:
help(mikeio)


Help on package mikeio:

NAME
    mikeio

PACKAGE CONTENTS
    base
    custom_exceptions
    data_utils
    dataarray
    dataset
    dfs
    dfs0
    dfs1
    dfs2
    dfs3
    dfsu
    dfsu_factory
    dfsu_layered
    dfsu_spectral
    dfsutil
    eum
    generic
    helpers
    interpolation
    pfs
    spatial (package)
    spectral_utils
    xyz

FUNCTIONS
    open(filename: str, **kwargs)
        Open a dfs/mesh file (and read the header)
        
        The typical workflow for small dfs files is to read all data
        with *mikeio.read* instead of using this function. For big files, however,
        it can be convenient to open the file first with *dfs=mikeio.open(...)* to
        inspect it's content (items, time and shape) and then decide what to
        read using dfs.read(...)
        
        Parameters
        ----------
        filename
            full path and file name to the dfs file.
        type : str, optional
            Dfs2 only. Additional information abo

In [50]:
nodes1 = gauges[gauges['Level Type']=='Node']['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
nodes2 = gauges[gauges['Level Type']=='Node']['Node2'].dropna().unique().flatten().tolist()
nodes = nodes1 + nodes2
#Convert all to string
nodes = [str(x) for x in nodes]
#Make set to clear duplicates
nodes = set(nodes)

pipes1 = gauges[gauges['Level Type'].str.contains('Link')]['Node1 (Or Pipe if pipe level)'].dropna().unique().flatten().tolist()
pipes2 = gauges.Pipe.dropna().unique().flatten().tolist()
pipes = pipes1 + pipes2
#Convert all to string
pipes = [str(x) for x in pipes]
#Make set to clear duplicates
pipes = set(pipes)

In [97]:
from mikeio.dfs0 import Dfs0
dfs0_folder = r"\\prdsynfile01\lws_modelling\SEWER_AREA_MODELS\NSSA\02_MODEL_COMPONENTS\06_FLOW MONITORING\10. Dfs0s\Non-PS"

dfs = Dfs0()
res = mikeio.read(dfs0_folder + '\\' 'HL14_20180614_20190403_5min.dfs0')
df = res.to_dataframe()
df.iloc[:,[0,6]]

,HL14_Flow_L,HL14_Level
2018-06-14 00:00:00,105.150002,-0.659200
2018-06-14 00:05:00,106.050003,-0.657905
2018-06-14 00:10:00,108.650002,-0.654010
2018-06-14 00:15:00,110.349998,-0.650690
2018-06-14 00:20:00,110.150002,-0.650770
...,...,...
2019-04-03 23:35:00,109.550003,-0.649750
2019-04-03 23:40:00,110.150002,-0.648520
2019-04-03 23:45:00,109.849998,-0.648040
2019-04-03 23:50:00,108.699997,-0.649165


In [123]:
f = 'HL14_20180614_20190403_5min.dfs0'

ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
gauge_id = re.split('_|.',f)[0]
ts['Gauge'] = gauge_id
ts.rename(columns={ts.columns[1]:'Flow'},inplace=True)
if len(ts.columns)>6:
    ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
else:
    ts['Level'] = np.nan
ts = ts[['Gauge','Flow','Level']]


In [158]:
gauges = []
first_dfs0 = True
for dfs0_folder in dfs0_folders:
    for f in os.listdir(dfs0_folder):
        if f[-5:]=='.dfs0':
            gauge_id = re.split(r'_|[.]',f)[0]
            ts = mikeio.read(dfs0_folder + '\\' + f).to_dataframe()
            ts['Gauge'] = gauge_id
            gauges.append(gauge_id)
            ts.rename(columns={ts.columns[0]:'Flow'},inplace=True)
            if len(ts.columns)>5:
                ts.rename(columns={ts.columns[6]:'Level'},inplace=True) 
            else:
                ts['Level'] = np.nan
            ts = ts[['Gauge','Flow','Level']]
            if first_dfs0 == True:
                measured = ts.copy()
            else:
                measured = pd.concat([measured,ts])
            first_dfs0 = False
            
        

In [159]:
gauges

['FA2',
 'GN14',
 'HL14',
 'HL18',
 'HL1',
 'HL33',
 'HL94',
 'HL95',
 'HL97',
 'HL9',
 'MK1',
 'MK20',
 'MK3',
 'NV13',
 'NV19',
 'NV1',
 'NV20',
 'NV21',
 'NV31',
 'NV45',
 'NV8',
 'NV93',
 'NV94',
 'NV9',
 'NVLS1',
 'NVLS5-450',
 'NVLS5-500',
 'NVLS5-Blowdown',
 'NVLS9',
 'TW3B-MP1',
 'TW3B-MP2',
 'GE1',
 'GE2',
 'GE3',
 'GE4',
 'GE5',
 'LY1',
 'LY2',
 'LY7']

In [172]:
gauge = 'HL14'
for period_spec in period_specs:
    start = periods[periods.Meter==gauge][period_spec + ' Start'].iloc[0]
    end = periods[periods.Meter==gauge][period_spec + ' End'].iloc[0]

In [173]:
end

Timestamp('2020-01-02 00:00:00')

In [161]:
periods

,Meter,Zone,Sub unit,Meter Status,Model,DWF Start,DWF End,WWF1 Start,WWF1 End,WWF2 Start,WWF2 End,WWF3 Start,WWF3 End,WWF4 Start,WWF4 End,WWF5 Start,WWF5 End,WWF6 Start,WWF6 End,No of WWF Calibration (excl Val).
217,FA2,FA2,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
218,GE1,GE1,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
219,GE2,GE2,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
220,GE3,GE3,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
221,GE4,GE5,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
222,GE5,GE5,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
223,GN14,GN14,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
224,HL1,HL1,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
225,HL14,HL14,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4
226,HL18,HL14,NaN,NaN,NSSA,2018-07-26,2018-07-30,2018-12-12,2018-12-15,2018-10-31,2018-11-03,2018-11-25,2018-11-28,2019-01-02,2019-01-05,2019-12-29,2020-01-02,2020-01-29,2020-02-02,4


In [67]:
results = []
first_round = True

for f in os.listdir(result_folder):
    if f[-6:]=='.res1d' and not 'ADDOUT' in f and not 'RR' in f and not 'UserSpecified' in f:
        resultdata = Res1D_v5(result_folder + '\\' + f)
        reaches = resultdata.resultData.Reaches

        print ("Importing network " + f + " at " + str(dt.datetime.now()))

#         missing_muids = set()

#         #Check for missing MUIDs in the result file
#         muid_checks = set()

#         rid_checks = set()
#         for i, reach in enumerate(reaches):
#             rid_checks.add(reach.Id[:(-1 * len(str(i)) - 1)])
#         for rid in network_import_specs["MUID_Extended"].values:
#             if not rid in rid_checks:
#                 muid_checks.add(rid)
#         if len(muid_checks)>0:
#             print('In Network_Import_Specs but not in re: ' + str(muid_checks))
#             break


        for i, reach in enumerate(reaches):
            rid = reach.Id[:(-1 * len(str(i)) - 1)]
            gridpoints = reach.GridPoints.Count

            if rid in pipes:

                print ("Importing pipe " + reach.Id + " at " + str(dt.datetime.now()))

                for di in reach.DataItems:

                    if di.Quantity.Id.lower() == 'discharge':


                        values = list(di.CreateTimeSeriesData(0))
                        flow_df = pd.DataFrame(index = resultdata.dfTimeStamp)
                        flow_df['ResultFile'] = f
#                         flow_df['MUID'] = network_import_specs[network_import_specs['MUID_Extended']==rid].iloc[0]['MUID']
                        flow_df['MUID'] = rid
                        flow_df['Discharge'] = values                          
                        flow_df['Discharge'] = flow_df['Discharge'] * 1000

                        if rolling_average == True:
                            flow_df['Discharge_Hourly'] = flow_df['Discharge'].rolling('1h').mean()


                        flow_df['DateTimeRef'] = flow_df.index

                        if first_round == True:
                            df_result = flow_df.copy()
                            first_round = False
                        else:
                            df_result = pd.concat([df_result,flow_df])
                        first_round = False


Importing network NSSA_DWF_Cal_2018-07-26_Base.res1d at 2022-10-14 10:49:51.980717
Importing pipe 40472-21 at 2022-10-14 10:49:51.980717
Importing pipe 41250-74 at 2022-10-14 10:49:51.984721
Importing pipe 41295-120 at 2022-10-14 10:49:51.987723
Importing pipe 41304-129 at 2022-10-14 10:49:51.991727
Importing pipe 43611-209 at 2022-10-14 10:49:51.994730
Importing pipe 43648-245 at 2022-10-14 10:49:51.997733
Importing pipe 43945-350 at 2022-10-14 10:49:52.001736
Importing pipe 43946-351 at 2022-10-14 10:49:52.004739
Importing pipe 43953-358 at 2022-10-14 10:49:52.008743
Importing pipe 43987-390 at 2022-10-14 10:49:52.011745
Importing pipe 43989-392 at 2022-10-14 10:49:52.014748
Importing pipe 43999-402 at 2022-10-14 10:49:52.018752
Importing pipe 47391-580 at 2022-10-14 10:49:52.021755
Importing pipe 47438-627 at 2022-10-14 10:49:52.025758
Importing pipe 49559-884 at 2022-10-14 10:49:52.029762
Importing pipe 49580-901 at 2022-10-14 10:49:52.032765
Importing pipe 49582-903 at 2022-10-14 

Importing network NSSA_WWF_Cal_2018-11-25_Base.res1d at 2022-10-14 10:49:55.021507
Importing pipe 40472-21 at 2022-10-14 10:49:55.021507
Importing pipe 41250-74 at 2022-10-14 10:49:55.026512
Importing pipe 41295-120 at 2022-10-14 10:49:55.031516
Importing pipe 41304-129 at 2022-10-14 10:49:55.036521
Importing pipe 43611-209 at 2022-10-14 10:49:55.040525
Importing pipe 43648-245 at 2022-10-14 10:49:55.045529
Importing pipe 43945-350 at 2022-10-14 10:49:55.050534
Importing pipe 43946-351 at 2022-10-14 10:49:55.054538
Importing pipe 43953-358 at 2022-10-14 10:49:55.058541
Importing pipe 43987-390 at 2022-10-14 10:49:55.063546
Importing pipe 43989-392 at 2022-10-14 10:49:55.067550
Importing pipe 43999-402 at 2022-10-14 10:49:55.072554
Importing pipe 47391-580 at 2022-10-14 10:49:55.078560
Importing pipe 47438-627 at 2022-10-14 10:49:55.083564
Importing pipe 49559-884 at 2022-10-14 10:49:55.088569
Importing pipe 49580-901 at 2022-10-14 10:49:55.092572
Importing pipe 49582-903 at 2022-10-14 

Importing pipe 56401-1605 at 2022-10-14 10:49:57.499781
Importing pipe 56405-1606 at 2022-10-14 10:49:57.505787
Importing pipe Link_15-1612 at 2022-10-14 10:49:57.512793
Importing network NSSA_WWF_Cal_2020-01-29_Base.res1d at 2022-10-14 10:49:58.122353
Importing pipe 40472-21 at 2022-10-14 10:49:58.122353
Importing pipe 41250-74 at 2022-10-14 10:49:58.129359
Importing pipe 41295-120 at 2022-10-14 10:49:58.136366
Importing pipe 41304-129 at 2022-10-14 10:49:58.142371
Importing pipe 43611-209 at 2022-10-14 10:49:58.149377
Importing pipe 43648-245 at 2022-10-14 10:49:58.155383
Importing pipe 43945-350 at 2022-10-14 10:49:58.162389
Importing pipe 43946-351 at 2022-10-14 10:49:58.168395
Importing pipe 43953-358 at 2022-10-14 10:49:58.176402
Importing pipe 43987-390 at 2022-10-14 10:49:58.182408
Importing pipe 43989-392 at 2022-10-14 10:49:58.188413
Importing pipe 43999-402 at 2022-10-14 10:49:58.194419
Importing pipe 47391-580 at 2022-10-14 10:49:58.201425
Importing pipe 47438-627 at 2022-1

In [69]:
for gauge in gauges:
    

,ResultFile,MUID,Discharge,DateTimeRef
2018-07-26 00:00:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.468840,2018-07-26 00:00:00
2018-07-26 00:05:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,16.175270,2018-07-26 00:05:00
2018-07-26 00:10:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.881900,2018-07-26 00:10:00
2018-07-26 00:15:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.589100,2018-07-26 00:15:00
2018-07-26 00:20:00,NSSA_DWF_Cal_2018-07-26_Base.res1d,40472,15.296860,2018-07-26 00:20:00
...,...,...,...,...
2020-02-01 23:40:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Link_15,0.003972,2020-02-01 23:40:00
2020-02-01 23:45:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Link_15,0.003972,2020-02-01 23:45:00
2020-02-01 23:50:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Link_15,0.003972,2020-02-01 23:50:00
2020-02-01 23:55:00,NSSA_WWF_Cal_2020-01-29_Base.res1d,Link_15,0.003972,2020-02-01 23:55:00
